In [1]:
import pandas as pd

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

## Create new features¶
As in Week 2, we consider features that are some transformations of inputs.

In [12]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

print(sales.shape)

(21613, 25)


# Learn regression weights with L1 penalty¶

Let us fit a model with all the features available, plus the features we just created above.

In [36]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [37]:
# print coefficient
coef = model_all.coef_
print(coef)

new_features = ['sqft_living', 
            'view', 'grade']

[     0.              0.              0.            134.43931396      0.
      0.              0.              0.              0.              0.
  24750.00458561      0.          61749.10309071      0.              0.
     -0.              0.        ]


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection.

QUIZ QUESTION: According to this list of weights, which of the features have been chosen?

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:

Split our sales data into 2 sets: training and test
Further split our training data into two sets: train, validation
Be very careful that you use seed = 1 to ensure you get the same answer!

In [11]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)
print(testing.shape)

(2217, 21)


Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [13]:
# Adding new features
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [17]:
import numpy as np
l1_penalty = np.logspace(1, 7, num=13)


In [42]:
RSS_validation = {}
for l in range(len(l1_penalties)):
    model_all = linear_model.Lasso(alpha=l1_penalty[l], normalize=True) # set parameters
    model_all.fit(training[all_features], training['price']) # learn weights
    predictions = model_all.predict(validation[all_features])
    RSS_validation[l]= ((predictions- validation['price'])**2).sum()

In [21]:
print(RSS_validation)

{0: 398213327300134.4, 1: 399041900253348.2, 2: 429791604072558.2, 3: 463739831045119.4, 4: 645898733633803.0, 5: 1222506859427156.8, 6: 1222506859427156.8, 7: 1222506859427156.8, 8: 1222506859427156.8, 9: 1222506859427156.8, 10: 1222506859427156.8, 11: 1222506859427156.8, 12: 1222506859427156.8}


In [43]:
RSS = pd.DataFrame(list(RSS_validation.values()))
RSS.columns =['RSS_validation']
RSS['l1_penalty']= l1_penalty

In [44]:
print(RSS)

    RSS_validation    l1_penalty
0     3.982133e+14  1.000000e+01
1     3.990419e+14  3.162278e+01
2     4.297916e+14  1.000000e+02
3     4.637398e+14  3.162278e+02
4     6.458987e+14  1.000000e+03
5     1.222507e+15  3.162278e+03
6     1.222507e+15  1.000000e+04
7     1.222507e+15  3.162278e+04
8     1.222507e+15  1.000000e+05
9     1.222507e+15  3.162278e+05
10    1.222507e+15  1.000000e+06
11    1.222507e+15  3.162278e+06
12    1.222507e+15  1.000000e+07


## QUIZ QUESTION. What was the best value for the l1_penalty?

In [45]:
model_all = linear_model.Lasso(alpha=1e1, normalize=True) # set parameters
model_all.fit(training[all_features], training['price']) # learn weights

Lasso(alpha=10.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

## QUIZ QUESTION Also, using this value of L1 penalty, how many nonzero weights do you have?

In [46]:
print(model_all.coef_)

[ -1.61445628e+04   3.73245384e+02   5.08412433e+04   6.17853560e+02
  -4.44113549e+04   7.85623065e-01  -7.01194765e+02  -0.00000000e+00
   5.01420046e+03   6.19488752e+05   3.80418557e+04   2.49987718e+04
   1.28716235e+05   0.00000000e+00   0.00000000e+00  -3.29383118e+03
   1.00573209e+01]


Now that you have selected an L1 penalty, compute the RSS on TEST data for the model with the best L1 penalty.

In [51]:
predictions = model_all.predict(testing[all_features])
RSS= ((predictions- testing['price'])**2).sum()
print("RSS of test: ", RSS/1e14)

RSS of test:  0.9846740255269882


## Quiz Question: Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero. A succinct way to do this is

In [54]:
print(np.count_nonzero(model_all.coef_) + np.count_nonzero(model_all.intercept_))

15


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

## 9 Explore small penalty

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

You are going to implement a simple, two phase procedure to achieve this goal:

Explore a large range of ‘l1_penalty’ values to find a narrow region of ‘l1_penalty’ values where models are likely to have the desired number of non-zero weights.
Further explore the narrow region you found to find a good value for ‘l1_penalty’ that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for ‘l1

In [60]:
l1_penalty = np.logspace(1e-6, 2, num=20)

RSS_validation2 = {}
for l in range(len(l1_penalty)):
    model_all = linear_model.Lasso(alpha=l1_penalty[l], normalize=True) # set parameters
    model_all.fit(training[all_features], training['price']) # learn weights
    predictions = model_all.predict(validation[all_features])
    RSS_validation2[l]= ((predictions- validation['price'])**2).sum()
    
RSS2 = pd.DataFrame(list(RSS_validation2.values()))
RSS2.columns =['RSS_validation']
RSS2['l1_penalty']= l1_penalty

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [61]:
print(RSS2)

    RSS_validation  l1_penalty
0     4.028148e+14    1.000002
1     4.026290e+14    1.274278
2     4.023995e+14    1.623780
3     4.021187e+14    2.069142
4     4.017799e+14    2.636656
5     4.013792e+14    3.359824
6     4.009188e+14    4.281339
7     4.003739e+14    5.455603
8     3.995390e+14    6.951937
9     3.986081e+14    8.858679
10    3.977272e+14   11.288391
11    3.968318e+14   14.384513
12    3.962538e+14   18.329823
13    3.964390e+14   23.357232
14    3.982155e+14   29.763532
15    4.028974e+14   37.926920
16    4.121826e+14   48.329320
17    4.246475e+14   61.584836
18    4.264782e+14   78.476007
19    4.297916e+14  100.000000


In [64]:
RSS2 = RSS2.sort_values(by=['RSS_validation'])
print(RSS2)

    RSS_validation  l1_penalty
12    3.962538e+14   18.329823
13    3.964390e+14   23.357232
11    3.968318e+14   14.384513
10    3.977272e+14   11.288391
14    3.982155e+14   29.763532
9     3.986081e+14    8.858679
8     3.995390e+14    6.951937
7     4.003739e+14    5.455603
6     4.009188e+14    4.281339
5     4.013792e+14    3.359824
4     4.017799e+14    2.636656
3     4.021187e+14    2.069142
2     4.023995e+14    1.623780
1     4.026290e+14    1.274278
0     4.028148e+14    1.000002
15    4.028974e+14   37.926920
16    4.121826e+14   48.329320
17    4.246475e+14   61.584836
18    4.264782e+14   78.476007
19    4.297916e+14  100.000000


 Assign 7 to the variable ‘max_nonzeros’.

In [66]:
model_all = linear_model.Lasso(alpha=l1_penalty[12], normalize=True) # set parameters
model_all.fit(training[all_features], training['price']) # learn weights
print(model_all.coef_)

[ -1.65839645e+04   2.73907653e+02   4.76495668e+04   5.42165492e+02
  -3.68569835e+04   5.76217834e-01  -5.68757895e+02   0.00000000e+00
   5.14820826e+03   6.11476292e+05   3.89819843e+04   2.20925618e+04
   1.27322937e+05   0.00000000e+00   0.00000000e+00  -3.30891413e+03
   7.86012279e+00]


In [72]:
coeff= pd.DataFrame(abs(model_all.coef_))
coeff.columns =['coef']
coeff['name'] = all_features
coeff = coeff.sort_values(by=['coef'])
print(coeff)

             coef              name
14       0.000000     sqft_basement
13       0.000000        sqft_above
7        0.000000            floors
5        0.576218          sqft_lot
16       7.860123      yr_renovated
1      273.907653   bedrooms_square
3      542.165492       sqft_living
6      568.757895     sqft_lot_sqrt
15    3308.914135          yr_built
8     5148.208263     floors_square
0    16583.964514          bedrooms
11   22092.561840         condition
4    36856.983494  sqft_living_sqrt
10   38981.984318              view
2    47649.566771         bathrooms
12  127322.937157             grade
9   611476.291541        waterfront


In [81]:
selected_features = coeff['name'][-7:]
print(selected_features.values)

['bedrooms' 'condition' 'sqft_living_sqrt' 'view' 'bathrooms' 'grade'
 'waterfront']


## Exploring large range of l1_penalty

For l1_penalty in np.logspace(1, 4, num=20):

Fit a regression model with a given l1_penalty on TRAIN data. Add "alpha=l1_penalty" and "normalize=True" to the parameter list.

In [84]:
l1_penalty = np.logspace(1, 4, num=20)

RSS_validation3 = {}
count=[]
for l in range(len(l1_penalty)):
    model_all = linear_model.Lasso(alpha=l1_penalty[l], normalize=True) # set parameters
    model_all.fit(training[all_features], training['price']) # learn weights
    predictions = model_all.predict(validation[all_features])
    RSS_validation3[l]= ((predictions- validation['price'])**2).sum()
    count.append(np.count_nonzero(model_all.coef_) + np.count_nonzero(model_all.intercept_))
    
RSS3 = pd.DataFrame(list(RSS_validation3.values()))
RSS3.columns =['RSS_validation']
RSS3['l1_penalty']= l1_penalty
RSS3['non_zero']=count
RSS3 = RSS3.sort_values(by=['RSS_validation'])
print(RSS3)

    RSS_validation    l1_penalty  non_zero
2     3.962109e+14     20.691381        15
1     3.968318e+14     14.384499        15
0     3.982133e+14     10.000000        15
3     3.982155e+14     29.763514        15
4     4.068773e+14     42.813324        13
5     4.246475e+14     61.584821        12
6     4.279063e+14     88.586679        11
7     4.353747e+14    127.427499        10
8     4.431072e+14    183.298071         7
9     4.541767e+14    263.665090         6
10    4.781330e+14    379.269019         6
11    5.313972e+14    545.559478         6
12    5.940433e+14    784.759970         5
13    6.740592e+14   1128.837892         3
14    8.026094e+14   1623.776739         3
15    1.061255e+15   2335.721469         2
16    1.222507e+15   3359.818286         1
17    1.222507e+15   4832.930239         1
18    1.222507e+15   6951.927962         1
19    1.222507e+15  10000.000000         1


Extract the weights of the model and count the number of nonzeros. Take account of the intercept as we did in #8, adding 1 whenever the intercept is nonzero. Save the number of nonzeros to a list.

In [83]:
model_all = linear_model.Lasso(alpha=l1_penalty[2], normalize=True) # set parameters
model_all.fit(training[all_features], training['price']) # learn weights
print(np.count_nonzero(model_all.coef_) + np.count_nonzero(model_all.intercept_))
print(model_all.coef_)


15
[ -1.67201164e+04   2.46161126e+02   4.67505582e+04   5.20597513e+02
  -3.47045876e+04   5.16827492e-01  -5.31184505e+02   0.00000000e+00
   5.18627794e+03   6.09205096e+05   3.92492514e+04   2.12654824e+04
   1.26929364e+05   0.00000000e+00   0.00000000e+00  -3.31337004e+03
   7.23386017e+00]


Out of this large range, we want to find the two ends of our desired narrow range of l1_penalty. At one end, we will have l1_penalty values that have too few non-zeros, and at the other end, we will have an l1_penalty that has too many non-zeros.

More formally, find:

The largest l1_penalty that has more non-zeros than ‘max_nonzeros’ (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)Store this value in the variable ‘l1_penalty_min’ (we will use it later)
The smallest l1_penalty that has fewer non-zeros than ‘max_nonzeros’ (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)Store this value in the variable ‘l1_penalty_max’ (we will use it later)
Hint: there are many ways to do this, e.g.:

Programmatically within the loop above
Creating a list with the number of non-zeros for each value of l1_penalty and inspecting it to find the appropriate boundaries.

Quiz Question: What values did you find for l1_penalty_min and l1_penalty_max?

14. Exploring narrower range of l1_penalty

We now explore the region of l1_penalty we found: between ‘l1_penalty_min’ and ‘l1_penalty_max’. We look for the L1 penalty in this range that produces exactly the right number of nonzeros and also minimizes RSS on the VALIDATION set.

For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):

Fit a regression model with a given l1_penalty on TRAIN data. As before, use "alpha=l1_penalty" and "normalize=True".
Measure the RSS of the learned model on the VALIDATION set
Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’. (Again, take account of the intercept when counting the number of nonzeros.)

In [90]:
l1_penalty_min = l1_penalty[7]
l1_penalty_max = l1_penalty[9]


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set
We will now explore the narrow region of l1_penalty values we found:

In [91]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Fit a regression model with a given l1_penalty on TRAIN data. As before, use `alpha=l1_penalty` and `normalize=True`.
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [92]:
RSS_validation4 = {}
count=[]
for l in range(len(l1_penalty)):
    model_all = linear_model.Lasso(alpha=l1_penalty_values[l], normalize=True) # set parameters
    model_all.fit(training[all_features], training['price']) # learn weights
    predictions = model_all.predict(validation[all_features])
    RSS_validation4[l]= ((predictions- validation['price'])**2).sum()
    count.append(np.count_nonzero(model_all.coef_) + np.count_nonzero(model_all.intercept_))
    
RSS4 = pd.DataFrame(list(RSS_validation4.values()))
RSS4.columns =['RSS_validation']
RSS4['l1_penalty']= l1_penalty_values
RSS4['non_zero']=count
RSS4 = RSS4.sort_values(by=['RSS_validation'])
print(RSS4)

    RSS_validation  l1_penalty  non_zero
0     4.353747e+14  127.427499        10
1     4.370092e+14  134.597898        10
2     4.382361e+14  141.768298         8
3     4.391589e+14  148.938697         8
4     4.400374e+14  156.109097         7
5     4.407775e+14  163.279496         7
6     4.415667e+14  170.449896         7
7     4.424064e+14  177.620295         7
8     4.432967e+14  184.790695         7
9     4.442398e+14  191.961094         7
10    4.452307e+14  199.131494         7
11    4.462689e+14  206.301894         6
12    4.471129e+14  213.472293         6
13    4.479982e+14  220.642693         6
14    4.489247e+14  227.813092         6
15    4.498925e+14  234.983492         6
16    4.509015e+14  242.153891         6
17    4.519524e+14  249.324291         6
18    4.530439e+14  256.494690         6
19    4.541767e+14  263.665090         6


In [95]:
model_all = linear_model.Lasso(alpha=l1_penalty_values[4], normalize=True) # set parameters
model_all.fit(training[all_features], training['price']) # learn weights
print(np.count_nonzero(model_all.coef_) + np.count_nonzero(model_all.intercept_))
print(model_all.coef_)

coeff= pd.DataFrame(abs(model_all.coef_))
coeff.columns =['coef']
coeff['name'] = all_features
coeff = coeff.sort_values(by=['coef'])

print('')
print(coeff['name'][-7:])

7
[ -0.00000000e+00  -0.00000000e+00   1.06108903e+04   1.63380252e+02
   0.00000000e+00  -0.00000000e+00  -0.00000000e+00   0.00000000e+00
   0.00000000e+00   5.06451687e+05   4.19600436e+04   0.00000000e+00
   1.16253554e+05   0.00000000e+00   0.00000000e+00  -2.61223488e+03
   0.00000000e+00]

16    yr_renovated
3      sqft_living
15        yr_built
2        bathrooms
10            view
12           grade
9       waterfront
Name: name, dtype: object
